# Creando Datos para Entrenamiento y Entrenando Modelos

**Por qué actualizar un modelo?**

- Los modelos estadísticos hacen predicciones basadas en los ejemplos con los que fueron entrenados.
- Un modelo será más preciso con ejemplos de su dominio.
- Para predecir categorías específicas de su problema, el modelo debe aprender sobre ellas.
- Esencial para *text classification*, muy útil para *entity recognition* y un poco menos crítico para *tagging* y *parsing*.

**Cómo funciona el entrenamiento?**

Si no inicializamos con una pipeline entrenada:

1. **Inicializamos** el modelo con pesos aleatorios.
2. **Predecimos** un batch de ejemplos con los pesos actuales, usando `nlp.update`.
3. **Compara** la predicción con las labels verdaderas y **calcula** cómo cambiar los pesos para mejorar las predicciones.
4. **Actualización** de los pesos.
5. Paso 2. spaCy continua llamando `nlp.update` para cada batch de exemplos hasta que el modelo deje de mejorar.

Después del entrenamiento, podemos guardar un modelo actualizado y usarlo en nuestra aplicación.

## Datos de Entrenamiento

* Los **datos de entrenamiento** son los ejemplos con los que queremos actualizar el modelo. El **texto** debe ser similar a lo que el modelo verá en el runtime y el **label** es lo que el modelo debe predecir (categoría o un entity span y su tipo).

* Los datos de entrenamiento:
  * Suelen ser creados por humanos que asignan etiquetas a los textos. Herramientas de anotación: `Prodigy`, `Rubrix`, etc. 
  * Esto es mucho trabajo, pero puede ser semiautomatizado, por ejemplo, usando la clase `Matcher`.

* Dividimos los datos en entrenamiento y prueba, en formatos `.spacy`.
  * A partir de objetos `DocBin`.
  * En algunos casos, es posible que ya tenga datos en un formato común, por ejemplo, `.conll`, `.conllu` o `.iob`. El comando `convert`de spaCy convierte automáticamente estos archivos al formato binario de spaCy:

* `spacy convert`: El comando para correr
* `./train.gold.conll`: Ruta al archivo
* `./corpus`: Carpeta donde será colocado el archivo `.spacy`

## Entrenamiento

**Qué es el config.cfg?**

* `config.cfg` es la "única fuente de la verdad" para todos los settings.
* Define cómo inicializar el objeto `nlp`, qué componentes de pipeline adicionar y cómo deben configurarse sus implementaciones de modelos internos. También incluye todas las configuraciones para el proceso de entrenamiento y cómo cargar los datos, incluidos los hiperparámetros.
* Ayudan a la reproducibilidad de tu entrenamiento.
* El config es agrupado en secciones y las secciones anidadas son definidas usando un punto. Por ejemplo: `[components.ner.model]` define la configuración para la implementación del modelo del named entity recognizer.
* Los archivos config también pueden hacer referencia a funciones de Python mediante la notación `@`. Por ejemplo, el tokenizador define una función de tokenizador registrada.

**Generando un config**

* spaCy puede generar un archivo config predeterminado para ti.
* El widget `quickstart` le permite generar una configuración de forma interactiva seleccionando el idioma y los componentes de pipeline que necesita, así como configuraciones opcionales de hardware y de optimización.
* Alternativamente, también puede usar el comando `init config` de spaCy:

* `init config`: el comando para correr
* `config.cfg`: la trayectoria de salida para el config generado.
* `--lang`: la clase de lenguaje que debe ser usado por del pipeline, por ejemplo: `es` para Español.
* `--pipeline`: los nombres de los componentes del pipeline separados por comas para incluir

**Entrenando un pipeline**

* Para entrenar un pipeline, todo lo que necesitas es el archivo `config.cfg` y los datos de entrenamiento y teste (archivos `.spacy`).
* La configuración `config` puede ser sobreescrita en la línea de comando.

* `train`: el comando para correr
* `config.cfg`: la trayectoria al archivo config
* `--output`: la trayectoria a la carpeta de salida para salvar el pipeline entrenado.
* `--paths.train`: anula con ruta a los datos de entrenamiento
* `--paths.dev`: anula con ruta a los datos de teste

* Cada paso sobre los datos durante el entrenamiento es llamado una "época".
* Dentro de cada época, spaCy genera los scores de precisión a cada 200 ejemplos (puedes cambiar la frecuencia en el config).
* Cada línea muestra la pérdida (loss) y precisión calculada en este punto durante el entrenamiento.
* La puntuación más interesante a tener en cuenta es la puntuación combinada de la última columna. Refleja la precisión con la que su modelo predijo las respuestas correctas en los datos de evaluación.
* El entrenamiento se ejecuta hasta que el modelo deja de mejorar y termina automáticamente.

**Cargando un pipeline entrenado**

* El último pipeline entrenado (`model-last`) y el pipeline con el mejor score (`model-best`) son salvados en el directorio de salida.
* Puedes cargar tu pipeline entrenada utilizando `spacy.load`.

**Empaquetando tu pipeline**

* `spacy package` crea un paquete de Python instalable que contiene tu pipeline. Después de la instalación, puedes cargar tu pipeline usando su nombre. spaCy automáticamente adiciona el código del lenguaje al nombre (`my_pipeline` pasará a ser `es_my_pipeline`).
* Fácil de versionar e desplegar (deploy).

* `/path/to/output/model-best`: Trayectoria de tu pipeline exportado
* `/packages`: Carpeta de salida.
* `my_pipeline`: Nombre opcional.
* `1.0.0`: Versión opcional.

# Creando un Modelo Personalizado para PoS

Utilizaremos el arquivo conllu de Petrolês: http://petroles.ica.ele.puc-rio.br/

In [1]:
import os
os.chdir("model_pos")

In [2]:
!pwd

/home/jessi/Documentos/Data_Science/spacy/model_pos


In [3]:
!python -m spacy convert ./train.conllu ./corpus -n 10

ℹ Grouping every 10 sentences into a document.
✔ Generated output file (66 documents): corpus/train.spacy


In [4]:
!python -m spacy convert ./dev.conllu ./corpus -n 10

ℹ Grouping every 10 sentences into a document.
✔ Generated output file (17 documents): corpus/dev.spacy


In [5]:
!python -m spacy init config ./config.cfg --lang pt --pipeline morphologizer

ℹ Generated config template specific for your use case
- Language: pt
- Pipeline: morphologizer
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
!cat ./config.cfg

[paths]
train = null
dev = null
vectors = null
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "pt"
pipeline = ["tok2vec","morphologizer"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.morphologizer]
factory = "morphologizer"

[components.morphologizer.model]
@architectures = "spacy.Tagger.v1"
nO = null

[components.morphologizer.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${components.tok2vec.model.encode.width}
attrs = ["NORM","PREFIX","SUFFIX","SHAPE"]
rows = [5000,2500,2500,2500]
include_static_vectors = false

[components.tok2vec.model.encode]
@architectu

In [7]:
!python -m spacy train ./config.cfg --output ./output --paths.train ./corpus/train.spacy --paths.dev ./corpus/dev.spacy

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-12-13 06:33:24,841] [INFO] Set up nlp object from config
[2021-12-13 06:33:24,854] [INFO] Pipeline: ['tok2vec', 'morphologizer']
[2021-12-13 06:33:24,859] [INFO] Created vocabulary
[2021-12-13 06:33:24,859] [INFO] Finished initializing nlp object
[2021-12-13 06:33:26,350] [INFO] Initialized pipeline components: ['tok2vec', 'morphologizer']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'morphologizer']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS MORPH...  POS_ACC  MORPH_ACC  SCORE 
---  ------  ------------  -------------  -------  ---------  ------
  0       0          0.00         344.92    38.65      32.42    0.36
  3     200        559.91       20735.52    86.16      84.75    0.85
  6     400        410.96        3760.20    86.73      85.18    0.86
  9     

In [8]:
!python -m spacy package ./output/model-best ./packages --name my_pipeline_morphologizer --version 1.0.0

ℹ Building package artifacts: sdist
✔ Loaded meta.json from file
output/model-best/meta.json
✔ Generated README.md from meta.json
✔ Successfully created package 'pt_my_pipeline_morphologizer-1.0.0'
packages/pt_my_pipeline_morphologizer-1.0.0
running sdist
running egg_info
creating pt_my_pipeline_morphologizer.egg-info
writing pt_my_pipeline_morphologizer.egg-info/PKG-INFO
writing dependency_links to pt_my_pipeline_morphologizer.egg-info/dependency_links.txt
writing entry points to pt_my_pipeline_morphologizer.egg-info/entry_points.txt
writing requirements to pt_my_pipeline_morphologizer.egg-info/requires.txt
writing top-level names to pt_my_pipeline_morphologizer.egg-info/top_level.txt
writing manifest file 'pt_my_pipeline_morphologizer.egg-info/SOURCES.txt'
reading manifest file 'pt_my_pipeline_morphologizer.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'pt_my_pipeline_morphologizer.egg-info/SOURCES.txt'
running check


creating pt_my_pipeline_mor

In [9]:
os.chdir("packages/pt_my_pipeline_morphologizer-1.0.0")

In [10]:
!pwd

/home/jessi/Documentos/Data_Science/spacy/model_pos/packages/pt_my_pipeline_morphologizer-1.0.0


In [11]:
!pip install dist/pt_my_pipeline_morphologizer-1.0.0.tar.gz

Processing ./dist/pt_my_pipeline_morphologizer-1.0.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for pt-my-pipeline-morphologizer: filename=pt_my_pipeline_morphologizer-1.0.0-py3-none-any.whl size=6179012 sha256=df45b46afba45f91bdc33169b7a5497a2d682f9745136bb7d373cc13143de18a
  Stored in directory: /home/jessi/.cache/pip/wheels/4f/17/e6/bd00b77bfb6dcc970b68e14672aae4f810e90d11c4867bfba3
Successfully built pt-my-pipeline-morphologizer
  Attempting uninstall: pt-my-pipeline-morphologizer
    Found existing installation: pt-my-pipeline-morphologizer 1.0.0
    Uninstalling pt-my-pipeline-morphologizer-1.0.0:
      Successfully uninstalled pt-my-pipeline-morphologizer-1.0.0


In [12]:
#!python -m spacy download pt_core_news_sm
#!python -m spacy download pt_core_news_md
#!python -m spacy download pt_core_news_lg

In [13]:
import spacy 
import re

nlp_sm = spacy.load("pt_core_news_sm")
nlp_md = spacy.load("pt_core_news_md")
nlp_lg = spacy.load("pt_core_news_lg")
nlp = spacy.load("pt_my_pipeline_morphologizer")

text = "Um dos fatores que determina a qualidade da gasolina pirolisada e de suas frações de destilação e "\
"hidrogenação na indústria de petróleo é o teor de dienos conjugados, assim como o das olefinas, "\
"de compostos aromáticos e ramificados." 

dic = {"dos": "de os", "da": "de o", "na": "em o", "das": "de o"}

for k, v in dic.items():
    text = re.sub(rf"\b{k}\b", v, text)            
    
print(text, end = "\n\n")    

print("MODELO SMALL PARA PORTUGUÉS")
doc = nlp_sm(text)    
print([(token.text, token.pos_) for token in doc])
print(len(doc[2].vector))

print("\nMODELO MÉDIO PARA PORTUGUÉS")
doc = nlp_md(text)    
print([(token.text, token.pos_) for token in doc])
print(len(doc[2].vector))

print("\nMODELO LARGE PARA PORTUGUÉS")
doc = nlp_lg(text)    
print([(token.text, token.pos_) for token in doc])
print(len(doc[2].vector))

print("\nMODELO CREADO")
doc = nlp(text)
print([(token.text, token.pos_) for token in doc])
print(len(doc[2].vector))

Um de os fatores que determina a qualidade de o gasolina pirolisada e de suas frações de destilação e hidrogenação em o indústria de petróleo é o teor de dienos conjugados, assim como o de o olefinas, de compostos aromáticos e ramificados.

MODELO SMALL PARA PORTUGUÉS
[('Um', 'NUM'), ('de', 'ADP'), ('os', 'DET'), ('fatores', 'NOUN'), ('que', 'PRON'), ('determina', 'VERB'), ('a', 'DET'), ('qualidade', 'NOUN'), ('de', 'ADP'), ('o', 'DET'), ('gasolina', 'NOUN'), ('pirolisada', 'VERB'), ('e', 'CCONJ'), ('de', 'ADP'), ('suas', 'DET'), ('frações', 'NOUN'), ('de', 'ADP'), ('destilação', 'NOUN'), ('e', 'CCONJ'), ('hidrogenação', 'NOUN'), ('em', 'ADP'), ('o', 'DET'), ('indústria', 'NOUN'), ('de', 'ADP'), ('petróleo', 'NOUN'), ('é', 'AUX'), ('o', 'DET'), ('teor', 'NOUN'), ('de', 'ADP'), ('dienos', 'NOUN'), ('conjugados', 'VERB'), (',', 'PUNCT'), ('assim', 'ADV'), ('como', 'ADP'), ('o', 'PRON'), ('de', 'ADP'), ('o', 'DET'), ('olefinas', 'NOUN'), (',', 'PUNCT'), ('de', 'ADP'), ('compostos', 'NOUN'

**References**

- https://prodi.gy/
- https://www.rubrix.ml/
- https://docs.rubrix.ml/en/stable/
- https://spacy.io/usage/training#quickstart
- https://spacy.io/usage/training#config
- https://spacy.io/usage/training#config-custom
- https://spacy.io/api/morphologizer